### **HugginFace - Transformers - Pregunta a tu PDF** ###

Vamos a crear una aplicación que nos permita consultar a ChatGPT sobre la información que le proporcionemos en un documento PDF.

## **Módulos:** ##

In [ ]:
!pip install PyPDF2
!pip install langchain
!pip install sentence-transformers
!pip install faiss-gpu
!pip install openai

import requests

Podemos trabajar con un PDF que tengamos en local o programar la descarga de un documento PDF disponible online.

Si queremos descargar el manual de Python, podemos ejecutar la siguiente celda. En caso de querer trabajar con un PDF local, no ejecutar la siguiente celda.

In [ ]:
URL = 'https://aprendepython.es/_downloads/907b5202c1466977a8d6bd3a2641453f/aprendepython.pdf'
doc_to_download = requests.get(URL)
pdf_file = open('aprendepython.pdf','wb')
pdf_file.write(doc_to_download.content)

## **Lectura de Fichero PDF** ##

In [3]:
from PyPDF2 import PdfReader

In [4]:
# Cargar fichero PDF
pdf_file_obj = open('aprendepython.pdf', 'rb')
pdf_reader = PdfReader(pdf_file_obj)

text = ''
for page in pdf_reader.pages:
  text += page.extract_text()

## **Crear Chunks** ##

Dado que no podemos pasarle a ChatGPT todo el texto de una vez, tenemos que dividirlo en trozos (Chunks) que sí podamos pasar como entrada de datos.

Para hacer este trabajo, vamos a hacer uso de la librería *langchain*. La función RecursiveCharacterTextSplitter permite dividir un texto de entrada en trozos, considerando si es posible tomar párrafos completos, si no, frases, si no, palabras. Permite escoger la longitud de los trozos a crear y si queremos que haya "overlap" (parte del final de cada trozo, estará también como inicio del trozo siguiente. El objetivo es permitir una mayor comprensión del documento de entrada).

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    length_function=len
)

In [7]:
chunks = text_splitter.split_text(text)

In [ ]:
print(len(chunks))
print(chunks[0])

## **Crear Embeddings:** ##

Para pasarle los textos de entrada al modelo, primero debemos convertirlos en valores numéricos. Para crear los Embeddings vamos a utilizar el modelo "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2" (modelo pre-entrenado para detección de paráfrasis multilingüe).

**Características principales:**

* Multilingüe: Funciona con más de 50 idiomas, por lo que puede comparar textos en diferentes lenguas.
* Detección de paráfrasis: Identifica si dos oraciones dicen lo mismo con palabras diferentes, incluso dentro de idiomas distintos.
* Aprendizaje MiniLM-L12: Se basa en la arquitectura Transformer, específicamente utilizando la variante MiniLM-L12, que es eficiente y flexible.
* Vectorización semántica: Representa oraciones como vectores de 384 dimensiones, permitiendo comparar el significado de manera numérica.


**Aplicaciones:**

* Búsqueda semántica: Encontrar documentos relevantes en grandes colecciones de texto, a pesar de las diferencias de redacción.
* Eliminación de duplicados: Identificar y eliminar textos redundantes en diferentes idiomas.
* Minería de texto: Analizar grandes conjuntos de datos textuales identificando relaciones semánticas entre oraciones.
* Chatbots multilingües: Desarrollar chatbots capaces de entender y responder en múltiples idiomas.

In [ ]:
# Dos tamaños
'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2' #  471M
'sentence-transformers/paraphrase-multilingual-mpnet-base-v2' # 1.11G

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

Una vez cargado el modelo, podemos usarlo para crear los embeddings de nuestros chunks.

In [12]:
from langchain.vectorstores import FAISS

In [13]:
knowledge_base = FAISS.from_texts(chunks, embeddings)

Hemos creado una base de datos local que contiene el conocimiento de nuestro documento PDF. Ahora, le podemos pedir a dicha BD que nos devuelva los N chunks que más relación semántica tengan con la consulta que queremos hacer al documento.

In [24]:
pregunta = '¿Qué es VSCode?'
docs = knowledge_base.similarity_search(pregunta, 3)

In [ ]:
docs

## **Preguntar al documento PDF** ##

Vamos a lanzar una consulta al documento PDF haciendo uso de ChatGPT. Para ello, necesaritamos contar con la [clave API de OpenIA](https://platform.openai.com/api-keys).

In [17]:
import os
from google.colab import userdata

# Añadimos la API key como variable de entorno (la tomamos de los Secretos de GoogleColab)
os.environ['OPENAI_API_KEY'] = userdata.get('OpenIA_Key')

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

In [23]:
llm = ChatOpenAI(model_name = 'gpt-3.5-turbo')
chain = load_qa_chain(llm, chain_type='stuff')

In [ ]:
pregunta = '¿Qué es VSCode?'
docs = knowledge_base.similarity_search(pregunta, 3)
respuesta = chain.run(input_documents=docs, question=pregunta)
print(f"Respuesta ChatGPT: {respuesta}")